In [1]:
import os
import pandas as pd
from ticker_download_predict_upload import DownloadPredictUpload

In [2]:
dpu = DownloadPredictUpload()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
tickers = ["I:SPX", "QQQ", "NVDA", "WMT", "TGT"]

In [4]:
long_df_filename = os.path.join("input", f"Year of Tickers {dpu.get_today_date()}.csv")
if os.path.exists(long_df_filename):
    long_df = pd.read_csv(long_df_filename).set_index("datetime").sort_index()
else:
    date_from = dpu.past_business_day(pd.Timestamp(dpu.get_today_date()), 253)
    date_to = dpu.past_business_day(pd.Timestamp(dpu.get_today_date()), 1).replace(
        hour=23, minute=59, second=59
    )
    print(date_from, date_to)
    long_df = dpu.get_tickers(tickers, date_from=date_from, date_to=date_to)
    long_df.to_csv(long_df_filename, index=True)
long_df

2024-03-26 00:00:00 2025-03-28 23:59:59
I:SPX. Acquired 254 so far. Sleeping 5 seconds...
QQQ. Acquired 508 so far. Sleeping 5 seconds...
NVDA. Acquired 762 so far. Sleeping 5 seconds...
WMT. Acquired 1016 so far. Sleeping 5 seconds...
TGT. Acquired 1270 so far. Sleeping 5 seconds...


,ticker,open,high,low,close,volume,vwap,transactions
datetime,,,,,,,,
2024-03-25 04:00:00,NVDA,93.941,96.766,93.5100,95.002,552047330.0,95.4480,1113327.0
2024-03-25 04:00:00,WMT,60.870,60.985,60.3600,60.570,14186627.0,60.5604,105344.0
2024-03-25 04:00:00,QQQ,443.550,446.260,442.5400,444.760,27779904.0,445.0143,304143.0
2024-03-25 04:00:00,TGT,170.000,173.580,169.7300,172.660,4262579.0,172.3819,59716.0
2024-03-25 05:00:00,I:SPX,5219.520,5229.090,5216.0900,5218.190,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2025-03-28 04:00:00,WMT,85.430,85.830,84.7000,85.150,14789418.0,85.1368,172741.0
2025-03-28 04:00:00,QQQ,479.810,480.520,468.0500,468.940,46399302.0,472.5380,501401.0
2025-03-28 04:00:00,NVDA,111.485,112.870,109.0701,109.670,229862293.0,110.1686,1847381.0
